In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from PIL import Image
import cv2

import time
import os
from tqdm.notebook import tqdm
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from sklearn.model_selection import KFold

In [5]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"khoonhonglee","key":"0fc2bae8a22e99e8378539d97ae13b79"}'}

In [6]:
!pip install -q kaggle

In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [8]:
!chmod 600 /root/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d bulentsiyah/semantic-drone-dataset

100% 3.89G/3.89G [02:03<00:00, 38.9MB/s]
100% 3.89G/3.89G [02:03<00:00, 33.7MB/s]


In [10]:
!unzip semantic-drone-dataset.zip

Archive:  semantic-drone-dataset.zip
  inflating: RGB_color_image_masks/RGB_color_image_masks/000.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/001.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/002.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/003.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/004.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/005.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/006.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/008.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/011.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/013.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/014.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/015.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/016.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/018.png  
  inflating: RGB_color_im

In [15]:
# Directory path of image and mask dataset
IMAGE_PATH = 'dataset/semantic_drone_dataset/original_images/'
MASK_PATH = 'dataset/semantic_drone_dataset/label_images_semantic/'

# 23 classes
NUM_CLASSES = 23 
IMAGE_SIZE = 256

In [16]:
# Create dataframe of image dataset
def create_df():
    name = []
    for dirname, _, filenames in os.walk(IMAGE_PATH):
        for filename in filenames:
            name.append(filename.split('.')[0])
    
    return pd.DataFrame({'id': name}, index = np.arange(0, len(name)))

In [17]:
# Data preprocessing
def preprocess_data(X, y, size):
    X_processed = []
    y_processed = []
    for i in tqdm(range(len(X))):
        img = cv2.imread(IMAGE_PATH + X[i] + '.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(MASK_PATH + y[i] + '.png', 0)
        img = cv2.resize(img, size)
        mask = cv2.resize(mask, size, interpolation=cv2.INTER_NEAREST)
        X_processed.append(img.reshape(-1, 256, 256, 3))
        y_processed.append(mask.reshape(-1, 256, 256, 1))
    X_processed = np.concatenate(X_processed, axis=0)
    y_processed = np.concatenate(y_processed, axis=0)
    return X_processed, y_processed

df = create_df()
print('Total Images : ', len(df))

Total Images :  400


In [18]:
# Split data into train & test set
# X_trainval, X_test = train_test_split(df['id'].values, test_size=0.3, random_state=19)
# X_train, X_val = train_test_split(X_trainval, test_size=0.15, random_state=19)

# print('Train Size   : ', len(X_train))
# print('Val Size     : ', len(X_val))
# print('Test Size    : ', len(X_test))

In [19]:
# define the desired size of the resized images
size = (256, 256)

# Preprocess the data
X_processed, y_processed = preprocess_data(df['id'].values, df['id'].values, size)
# X_val_processed, y_val_processed = preprocess_data(X_val, X_val, size)
# X_test_processed, y_test_processed = preprocess_data(X_test, X_test, size)

  0%|          | 0/400 [00:00<?, ?it/s]

In [20]:
# Check the shape of X_processed & y_processed
print(X_processed.shape)
print(y_processed.shape)

(400, 256, 256, 3)
(400, 256, 256, 1)


In [21]:
# perform semantic segmentation on one of the images in the dataset
def predictMask(image_id):
    # image_id = '513'
    img = cv2.imread(IMAGE_PATH + image_id + '.jpg')
    # img = cv2.imread("btest2.jpg")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(MASK_PATH + image_id + '.png', 0)

    img_resized = cv2.resize(img, size)
    mask_resized = cv2.resize(mask, size, interpolation=cv2.INTER_NEAREST)

    img_processed = img_resized.reshape(-1, 3)
    
    return img_processed

In [22]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report,ConfusionMatrixDisplay, roc_curve
from keras.metrics import MeanIoU
from keras.utils import to_categorical
from keras import backend as K
import statistics
import seaborn as sns

def evaluateMask(true_mask, predicted_mask): 
    # Flatten the masks to 1D arrays
    true_mask = to_categorical(true_mask.flatten(), num_classes=23)
    predicted_mask = to_categorical(predicted_mask.flatten(), num_classes=23)
    
    # Mean Iou Score
    mean_iou = MeanIoU(num_classes=23)
    mean_iou.update_state(true_mask, predicted_mask)
    iou_score = mean_iou.result().numpy()
    
    # Confusion matrix
    cm = confusion_matrix(np.argmax(true_mask, axis=1), np.argmax(predicted_mask, axis=1), labels=range(23))
    
    # Calculate the metrics
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    tn = np.sum(cm) - (tp + fp + fn)

    sns.heatmap(cm, annot=True)

    precision = tp / (tp + fp)
    precision_mean = np.mean(np.nan_to_num(precision))
    recall = tp / (tp + fn)
    recall_mean = np.mean(np.nan_to_num(recall))
    accuracy = np.sum(tp) / np.sum(cm)
    f1_score = 2 * precision * recall / (precision + recall)
    f1_score_mean = np.mean(np.nan_to_num(f1_score))
    dice_coefficient = (2 * tp) / (2 * tp + fp + fn)
    dice_coefficient_mean = np.mean(np.nan_to_num(dice_coefficient))
    
    # Print the results
    print("Classification Report:\n", classification_report(np.argmax(true_mask, axis=1), np.argmax(predicted_mask, axis=1)))
    print("Mean IoU score:", iou_score)
    print("Recall:", recall_mean)
    print("Precision:", precision_mean)
    print("Accuracy:", accuracy)
    print("F1 score:", f1_score_mean)
    print("Dice coefficient:", dice_coefficient_mean)

    # Plot the confusion matrix
    sns.heatmap(cm, xticklabels=['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7'
                                 , 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14'
                                 , 'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 
                                 'P21', 'P22'], 
                yticklabels=['A0', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7'
                            , 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15'
                            , 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22'],
    annot=True, fmt='d', annot_kws={'fontsize':6}, cmap="YlGnBu")

In [23]:
def convolution_block(
    block_input,
    num_filters=128,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

In [24]:
def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    resnet50 = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)


model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)
model.summary()

94765736/94765736 [==============================] - 3s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                             

In [25]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights = True)

In [26]:
def create_model():
    return DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)

In [31]:
BATCH_SIZE = 4
EPOCHS = 200
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def cross_validation(num_folds):
    kf = KFold(n_splits=num_folds)
    cv_scores, model_history = list(), list()

    for train, test in kf.split(X_processed, y_processed):
        cv_model = None
        cv_model = create_model()
        cv_model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss=loss,
            metrics=["accuracy"],
        )

        X_train, X_test = X_processed[train], X_processed[test]
        y_train, y_test = y_processed[train], y_processed[test]

        train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
        validation_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
        train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=False)
        validation_dataset = validation_dataset.batch(BATCH_SIZE, drop_remainder=False)
        cv_model.save_weights('cv_deeplab.h5')
        history = cv_model.fit(train_dataset,validation_data=validation_dataset, epochs=EPOCHS, batch_size=8, callbacks=[es])
        _, val_acc = cv_model.evaluate(validation_dataset, verbose = 1)
        cv_model.load_weights('cv_deeplab.h5')
        print('>%.3f' % val_acc)
        cv_scores.append(val_acc)
        model_history.append(cv_model)
    return np.mean(cv_scores), np.std(cv_scores)

In [32]:
mean, std = cross_validation(5)

Epoch 1/200
80/80 [==============================] - 44s 107ms/step - loss: 1.6749 - accuracy: 0.5307 - val_loss: 91.5452 - val_accuracy: 0.0239
Epoch 2/200
80/80 [==============================] - 7s 86ms/step - loss: 1.2052 - accuracy: 0.6423 - val_loss: 3.7623 - val_accuracy: 0.1997
Epoch 3/200
80/80 [==============================] - 7s 88ms/step - loss: 1.0005 - accuracy: 0.6920 - val_loss: 2.3092 - val_accuracy: 0.4132
Epoch 4/200
80/80 [==============================] - 7s 89ms/step - loss: 0.8315 - accuracy: 0.7436 - val_loss: 1.3638 - val_accuracy: 0.6109
Epoch 5/200
80/80 [==============================] - 7s 89ms/step - loss: 0.7214 - accuracy: 0.7789 - val_loss: 1.2600 - val_accuracy: 0.5892
Epoch 6/200
80/80 [==============================] - 7s 89ms/step - loss: 0.6700 - accuracy: 0.7908 - val_loss: 1.3426 - val_accuracy: 0.6229
Epoch 7/200
80/80 [==============================] - 7s 88ms/step - loss: 0.5592 - accuracy: 0.8258 - val_loss: 1.0716 - val_accuracy: 0.6723
Epo

In [33]:
print('\nCross Validation with 5 fold')
print('---------------------------------')
print('Estimated Accuracy %.4f (%.6f)' % (mean, std))


Cross Validation with 5 fold
---------------------------------
Estimated Accuracy 0.8052 (0.016146)


In [34]:
mean, std = cross_validation(7)
print('\nCross Validation with 7 fold')
print('---------------------------------')
print('Estimated Accuracy %.4f (%.6f)' % (mean, std))

Epoch 1/200
86/86 [==============================] - 45s 120ms/step - loss: 1.7011 - accuracy: 0.5337 - val_loss: 20.4232 - val_accuracy: 0.0889
Epoch 2/200
86/86 [==============================] - 7s 87ms/step - loss: 1.2237 - accuracy: 0.6367 - val_loss: 6.2751 - val_accuracy: 0.3263
Epoch 3/200
86/86 [==============================] - 7s 85ms/step - loss: 1.0194 - accuracy: 0.6911 - val_loss: 1.2541 - val_accuracy: 0.6360
Epoch 4/200
86/86 [==============================] - 8s 87ms/step - loss: 0.8882 - accuracy: 0.7307 - val_loss: 3.9650 - val_accuracy: 0.2538
Epoch 5/200
86/86 [==============================] - 7s 81ms/step - loss: 0.7565 - accuracy: 0.7673 - val_loss: 1.7889 - val_accuracy: 0.4461
Epoch 6/200
86/86 [==============================] - 7s 82ms/step - loss: 0.6341 - accuracy: 0.8043 - val_loss: 1.4523 - val_accuracy: 0.5277
Epoch 7/200
86/86 [==============================] - 7s 85ms/step - loss: 0.5801 - accuracy: 0.8205 - val_loss: 1.2408 - val_accuracy: 0.6104
Epo

In [35]:
mean, std = cross_validation(9)
print('\nCross Validation with 9 fold')
print('---------------------------------')
print('Estimated Accuracy %.4f (%.6f)' % (mean, std))

Epoch 1/200
89/89 [==============================] - 42s 95ms/step - loss: 1.5785 - accuracy: 0.5544 - val_loss: 7.0868 - val_accuracy: 0.1501
Epoch 2/200
89/89 [==============================] - 7s 82ms/step - loss: 1.1490 - accuracy: 0.6526 - val_loss: 4.3133 - val_accuracy: 0.2013
Epoch 3/200
89/89 [==============================] - 7s 82ms/step - loss: 0.9801 - accuracy: 0.6948 - val_loss: 2.0667 - val_accuracy: 0.4621
Epoch 4/200
89/89 [==============================] - 7s 82ms/step - loss: 0.8041 - accuracy: 0.7526 - val_loss: 2.6218 - val_accuracy: 0.4117
Epoch 5/200
89/89 [==============================] - 7s 82ms/step - loss: 0.6823 - accuracy: 0.7894 - val_loss: 1.4044 - val_accuracy: 0.5572
Epoch 6/200
89/89 [==============================] - 7s 83ms/step - loss: 0.5803 - accuracy: 0.8207 - val_loss: 0.8266 - val_accuracy: 0.7406
Epoch 7/200
89/89 [==============================] - 8s 85ms/step - loss: 0.5010 - accuracy: 0.8441 - val_loss: 0.7564 - val_accuracy: 0.7624
Epoch